In [63]:
import math 
import io
import os
import torch
from torchvision import transforms
from torchvision.utils import save_image
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pillow_avif

from pytorch_msssim import ms_ssim
from compressai.zoo import bmshj2018_hyperprior,mbt2018_mean,mbt2018,cheng2020_attn,cheng2020_anchor,bmshj2018_factorized

from ipywidgets import interact,widgets
from skimage.metrics import structural_similarity as ssim
import cProfile
import lpips
loss_alex=lpips.LPIPS(net="alex")
loss_vgg=lpips.LPIPS(net="vgg")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressAIenv\Lib\site-packages\lpips\weights\v0.1\alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressAIenv\Lib\site-packages\lpips\weights\v0.1\vgg.pth


In [70]:
device="cuda" if torch.cuda.is_available() else "cpu"
print("Device:",device)

dataset_path=r'C:\Swapnil\Narrowband_DRONE\Drone_dataset_small_1'

Device: cuda


In [71]:
def DL_model(model_name,quality,metric):
    """
    I/P: get the model_name,quality,metric
    O/P: return the model
    """
    #check the model name is valid or not
    if model_name=="bmshj2018_hyperprior":
        if quality>8 or quality<1 or metric not in ["mse","ms-ssim"]:
            return 
        else:
            net=bmshj2018_hyperprior(quality=quality,metric=metric,pretrained=True).eval().to(device)
            return net
    elif model_name=="mbt2018":
        if quality>8 or quality<1 or metric not in ["mse","ms-ssim"]:
            return 
        else:
            net=mbt2018(quality=quality,metric=metric,pretrained=True).eval().to(device)
            return net
    elif model_name=="mbt2018_mean":
        if quality>8 or quality<1 or metric not in ["mse","ms-ssim"]:
            return 
        else:
            net=mbt2018_mean(quality=quality,metric=metric,pretrained=True).eval().to(device)
            return net

    elif model_name=="cheng2020_attn":
        if quality>6 or quality<1 or metric not in ["mse","ms-ssim"]:
            return 
        else:
            net=cheng2020_attn(quality=quality,metric=metric,pretrained=True).eval().to(device)
            return net
    elif model_name=="cheng2020_anchor":
        if quality>6 or quality<1 or metric not in ["mse","ms-ssim"]:
            return 
        else:
            net=cheng2020_anchor(quality=quality,metric=metric,pretrained=True).eval().to(device)
            return net
    else:
        print("Invalid model name requested")
        return 



In [97]:
model_name="bmshj2018_hyperprior"
quality=8
metric="ms-ssim"
net=DL_model(model_name,quality,metric)

print(f"Encoder Parameters :{sum(p.numel() for p in net.parameters())}")


Encoder Parameters :11816323


In [130]:
#PyTorch Model quantization
model=bmshj2018_factorized(quality=8,metric="ms-ssim")
import torch.quantization as quant
from torch.ao.quantization import QuantStub, DeQuantStub
print(model)
#model.eval()
#model.qconfig = torch.ao.quantization.default_qconfig
#print(model.qconfig)
#torch.ao.quantization.prepare(model, inplace=True)
#model.to(device)

#Quantization attempt failed 

FactorizedPrior(
  (entropy_bottleneck): EntropyBottleneck(
    (likelihood_lower_bound): LowerBound()
    (matrices): ParameterList(
        (0): Parameter containing: [torch.float32 of size 320x3x1]
        (1): Parameter containing: [torch.float32 of size 320x3x3]
        (2): Parameter containing: [torch.float32 of size 320x3x3]
        (3): Parameter containing: [torch.float32 of size 320x3x3]
        (4): Parameter containing: [torch.float32 of size 320x1x3]
    )
    (biases): ParameterList(
        (0): Parameter containing: [torch.float32 of size 320x3x1]
        (1): Parameter containing: [torch.float32 of size 320x3x1]
        (2): Parameter containing: [torch.float32 of size 320x3x1]
        (3): Parameter containing: [torch.float32 of size 320x3x1]
        (4): Parameter containing: [torch.float32 of size 320x1x1]
    )
    (factors): ParameterList(
        (0): Parameter containing: [torch.float32 of size 320x3x1]
        (1): Parameter containing: [torch.float32 of size 

In [129]:

#images=file_handling(dataset_path)
#image=Image.open(os.path.join(dataset_path,images[0]))
#x=transforms.ToTensor()(image).unsqueeze(0).to(device)
#with torch.no_grad():
#    out_net_=model.forward(x)
#out_net_['x_hat'].clamp(0,1)
#export_name=images[0].split('.')[0]+"QUANT_c.webp"
#export_name=os.path.join(dataset_path,export_name)
#save_image(out_net_['x_hat'],export_name)

In path: C:\Swapnil\Narrowband_DRONE\Drone_dataset_small_1
Files: 6
Images are in format: jpg count: 4


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [128]:
#pip install tensorflow

In [133]:

import torch.onnx

# Assuming you have a pre-trained model instance `model`
# and a way to get the input shape (check model documentation)
def get_input_shape(model):
  # Replace this with your logic to get the input shape
  # based on the model definition or documentation
  return (1, 3, 3840, 2160)  # Example 4K color image

# Create dummy input based on the retrieved shape
# Assuming input shape is (channels, height, width)
input_shape = get_input_shape(model)  # Replace with your logic
dummy_input = torch.randn(3, *input_shape[1:])  # Remove the first dimension (1)


try:
  # Attempt ONNX export
  torch.onnx.export(model, dummy_input, "model.onnx", opset_version=12)
  print("Successfully exported model to model.onnx")
except RuntimeError as e:
  print(f"Error during ONNX export: {e}")
  # Handle potential errors during export (e.g., unsupported ops)

# Optional: Convert ONNX to TFLite (if export successful)
try:
  import tensorflow as tf

  converter = tf.lite.TFLiteConverter.from_keras_model(None)  # No Keras model assumed
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.inference_input_types = [dummy_input.dtype]  # Set input type

  # Load the ONNX model (if export succeeded)
  with open("model.onnx", "rb") as f:
    converter.graph_def = f.read()

  tflite_model = converter.convert()

  # Save the TFLite model
  with open("model.tflite", "wb") as f:
    f.write(tflite_model)
  print("Successfully converted model to model.tflite")
except Exception as e:
  print(f"Error during TFLite conversion: {e}")
  # Handle potential errors during conversion (e.g., unsupported ops)


Error during ONNX export: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1194393600 bytes.
Error during TFLite conversion: No module named 'distutils'


In [ ]:
from enum import Enum

class InputType(Enum):
  FLOAT32 = "FLOAT32"
  UINT8 = "UINT8"

# Specify the input type based on your TFLite model (FLOAT32 or UINT8)
input_type = InputType.FLOAT32

# Load the TFLite model
interpreter = torch.jit.load("model.tflite")

# Define the input tensor (replace with your actual input data)
input_tensor = torch.randn(1, *input_shape)
if input_type == InputType.UINT8:
  input_tensor = input_tensor.quantize(dtype=torch.uint8)

# Set the input tensor
interpreter.set_input_tensor(0, input_tensor)

# Run inference
interpreter.invoke()

# Get the output tensor
output_tensor = interpreter.get_output_tensor(0)

# Do something with the output (e.g., print, further processing)
print(output_tensor)

In [74]:

def file_handling(path):

    files=os.listdir(path)
    print("In path:",path)
    print("Files:",len(files))
    images=[]
    image_formats=[".jpg",".jpeg",".png",".webp",".avif"]
    for file in files:
        if file.lower().endswith(tuple(image_formats)):
            images.append(file)
    if len(images)!=0:
        print("Images are in format:",images[0].split('.')[1],"count:",len(images))
    return images

file_handling(dataset_path)

In path: C:\Swapnil\Narrowband_DRONE\Drone_dataset_small_1
Files: 5
Images are in format: jpg count: 3


['sample.jpg', 'Test_img.jpg', 'tokyo_tower.jpg']

In [102]:

def resize_to_FHD(img):
    # Open the image using Pillow
    #img = Image.open(img_path)
    resized_img = img.resize((1920, 1080))
    return resized_img
        


def compress(dataset_path,model,quality,metric):
    """
        Results of first stage compression are saved as the .webp image
    """
    #checking the input parameters
    if model=="cheng2020_attn" or model=="cheng2020_anchor":
        if quality>6 or quality <1 or metric not in ["mse","ms-ssim"]:
            print("Invalid quality or metric value")
            return 
    elif quality>8 or quality <1 or metric not in ["mse","ms-ssim"]:
        print("Invalid quality or metric value")
        return 

    images=file_handling(dataset_path) #get the images list for the compression 
    export_path=os.path.join(dataset_path,"compressed")
    
    try:
        os.mkdir(export_path)
    except:
        print("Compressed directory already exists")
        

    for image in images:
        #load the image using Pillow
        print("For Image:",image)
        img=Image.open(os.path.join(dataset_path,image))
        

        #transform the loaded image to the tensor
        x=transforms.ToTensor()(img).unsqueeze(0).to(device)
        
        #perform compression using the DL model
        try:
            with torch.no_grad():
                out_net=net.forward(x)
                obj=out_net['x_hat'] #save the output of the network
            obj.clamp(0,1)
        except:
            print("Try again using resize")
            img=resize_to_FHD(img)
            x=transforms.ToTensor()(img).unsqueeze(0).to(device)
            try:
                with torch.no_grad():
                    out_net=net.forward(x)
                    obj=out_net['x_hat'] #save the output of the network
                obj.clamp(0,1)
            except:
                print("Failed again")
                return 
            

        #save the output of the network as the image using the save_image method
        name=image.split('.')[0]+"_c.webp"
        name_export=os.path.join(export_path,name)
        save_image(obj,name_export)

    return export_path
        

export_path=compress(dataset_path,net,8,"ms-ssim")
        


    

In path: C:\Swapnil\Narrowband_DRONE\Drone_dataset_small_1
Files: 5
Images are in format: jpg count: 3
Compressed directory already exists
For Image: sample.jpg
For Image: Test_img.jpg
Try again using resize
For Image: tokyo_tower.jpg
Try again using resize


In [103]:
def compress_stage_2(export_path):

    images=file_handling(export_path)

    for image in images:
        img_path=os.path.join(export_path,image)
        img=Image.open(img_path)
        #now export the image as the avif file
        img_name=image.split('.')[1]+"_c.avif"
        img_path_export=os.path.join(img_path,img_name)
        img.save(img_path,format="AVIF",quality=50)
        

compress_stage_2(export_path)

In path: C:\Swapnil\Narrowband_DRONE\Drone_dataset_small_1\compressed
Files: 3
Images are in format: webp count: 3
